In [25]:
from lime.lime_text import LimeTextExplainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch
import scipy as sp
import numpy as np

In [ ]:
# Note: This is for non-deep learning approaches

# Prepare everything before interpretability

# Load the dataset

# Transform the features (before it enters the model)

# Create the model and the pipeline

# LIME

This section is for intrepreting models, using LIME

In [16]:
# For loading
model_name = "nateraw/bert-base-uncased-emotion"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, output_hidden_states=True)
labels = sorted(model.config.label2id, key=model.config.label2id.get)

class_names = labels

In [33]:
def get_proba_scores(x):

    # Tokenize
    tv = torch.tensor(
        [
            tokenizer.encode(v, padding="max_length", max_length=128, truncation=True)
            for v in x
        ]
    )

    # Create attention mask
    attention_mask = (tv != 0).type(torch.int64)

    # Get the output logits and convert to scores
    outputs = model(tv, attention_mask=attention_mask)[0].detach().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    print(scores[0])
    return None
    val = sp.special.logit(scores)

    return val

sample = ['I really did not like what I saw and I hated every moment of it.']
print(labels)
get_proba_scores(sample)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
[0.01966359 0.18398522 0.01423227 0.6661366  0.0985421  0.01744025]


In [12]:
# Create the function to fetch the probabiltiy scores
def get_probability_scores(text):

    # Tokenizer
    tokenized = tokenizer(text, return_tensors="pt", padding=True)

    # Get output from model
    output = model(**tokenized)
    return None

    # Get the probabilities
    ...

sample_text = 'surprising increase in revenue in spite of decrease in market share'
get_probability_scores(sample_text)

# Create the lime text explainer object

# Create

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.5818, -0.3155, -0.1592,  ..., -0.3976,  0.1447,  0.0584],
         [-0.4463, -0.6126, -0.5334,  ..., -0.3823,  0.5842, -0.0931],
         [-1.2740, -0.4262,  0.5292,  ..., -0.8582,  0.1954, -0.9494],
         ...,
         [ 0.2317, -0.3000,  0.3801,  ..., -0.5091,  0.2487,  0.1577],
         [ 0.0126, -0.6731,  0.2026,  ..., -0.4416,  0.2002,  0.1120],
         [ 0.6529,  0.0146, -0.2628,  ...,  0.1625, -0.3500, -0.3292]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-9.0001e-01, -3.9651e-01, -8.2179e-01,  7.6373e-01,  5.4891e-01,
         -1.5884e-01,  8.9067e-01,  2.4533e-01, -6.6508e-01, -9.9995e-01,
         -4.6959e-01,  9.0916e-01,  9.6788e-01,  4.3327e-01,  8.8402e-01,
         -7.8692e-01, -6.5600e-01, -5.1406e-01,  3.3625e-01, -6.8360e-01,
          7.1572e-01,  9.9992e-01,  6.2722e-02,  4.1407e-01,  4.3372e-01,
          9.6215e-01, -7.4032e-01,  9.0022e-01,  9.3716e-01,  6.560

# SHAP

This section is for interpreting models using SHAP